<a href="https://colab.research.google.com/github/edoardorossi1995/Tesi/blob/main/Average_Reward_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pseudocodice 
###Algoritmo di tipo Q-learning, off policy, a task non episodico, che approssima la funzione valore con una rete neurale convessa, con un framework di tipo average reward.






#Drive Setting


In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB == True:
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd /content/gdrive/MyDrive/Tesi
  #!ls

import warnings
import pickle
import sys
if IN_COLAB == True:
  sys.path.insert(0, '/content/gdrive/MyDrive/Tesi/functions')
  sys.path.insert(0, '/content/gdrive/MyDrive/Tesi')

warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import Dense, Lambda
from keras.activations import exponential, relu
from keras import backend as K
from scipy.optimize import minimize

from pkl import store_data, load_data
from functions.compressor import compress

import random
from collections import defaultdict

Mounted at /content/gdrive
/content/gdrive/MyDrive/Tesi


# Import Anagrafica e CTA

In [3]:
#import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt

CTA30_def = ['/content/gdrive/MyDrive/Tesi/CTA30/DF_clean/clean_CTA_30_def.csv']
df_def = pd.read_csv(CTA30_def[0])

CTA30_18 = ['/content/gdrive/MyDrive/Tesi/CTA30/DF_clean/clean_CTA_30_2018.csv']
df = pd.read_csv(CTA30_18[0])
df['VLV_RAF_FDBK_00'] = df['VLV_RAF_FDBK_00'] / 100
df['VLV_SUR_FDBK_00'] = df['VLV_SUR_FDBK_00'] / 100
scaler = MinMaxScaler()
act_f_m = df.loc[:,'F_M_FDBK_00']
a_f_m_reshaped = np.array(act_f_m).reshape(-1, 1)
a_f_m_scaled = scaler.fit_transform(a_f_m_reshaped)

pca_df = compress(df)

In [4]:
print("Anagrafica shape:",df_def.shape,"\nDataframe shape:", df.shape,"\nPca Dataframe shape:", pca_df.shape)

Anagrafica shape: (76, 2) 
Dataframe shape: (141811, 76) 
Pca Dataframe shape: (141811, 7)


In [5]:
df_def

,pointColName,pointName
0,AUTOMAN_SERR_CMD_00,Automatico Manuale Serrande Comando
1,AUTOMAN_VLV_CMD_00,Automatico Manuale Valvole Comando
2,AUTOMAN_M_CMD_00,Automatico Manuale Mandata Comando
3,AUTOMAN_R_CMD_00,Automatico Manuale Ripresa Comando
4,LOCREM_CMD_00,Selettore Locale Remoto Comando
...,...,...
71,VOLT_R_00,Tensione Ripresa
72,T_M_SP_00,Temperatura Aria Mandata Setpoint
73,T_R_SP_00,Temperatura Aria Ritorno Setpoint
74,T_AMB_04,Temperatura Aria Ambiente Media


# Rete Neurale

Log-sum-exp neural networks and posynomial
models for convex and log-log-convex data 

(Calafiore, Gaubert, Possieri)

In [ ]:
n = load_data('*.pkl')
action_vector_dimension = 2

# Funzione di attivazione logatirmica (ultimo strato)
def log_act(x):
  return K.log(x+10)


# Architettura rete neurale
input_dim = n['n_components']+action_vector_dimension
inner_dim_1 = 32
output_dim = 1

print("input dim: ", input_dim)
print("inner layer dim: ", inner_dim_1)
print("output dim: ", output_dim)

# Costruzione strati
rete_neurale = Sequential()
rete_neurale.add(Dense(units = inner_dim_1, activation = exponential, input_dim = input_dim))
rete_neurale.add(Dense(units = output_dim, activation=log_act)) 

rete_neurale.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

# Definisco funzione obiettivo per la massimizzazione della predizione 

def objective(x):
    pca_s_prime = pca_df.loc[s_prime,:]
    nn_input_prime = np.concatenate((pca_s_prime.values.reshape(7,1), x[0].reshape(1,1), x[1].reshape(1,1)), axis=0)
    nn_input_prime = np.reshape(nn_input_prime, (1,input_dim))
    funct = rete_neurale.predict(nn_input_prime, verbose=0)[0][0]
    return -funct

input dim:  9
inner layer dim:  32
output dim:  1


In [7]:
(df.loc[5000:6000,'PW_ATT_M_00'])

5000    19.8775
5001    19.7422
5002    19.7584
5003    19.7964
5004    19.8194
         ...   
5996    20.0913
5997    20.0210
5998    20.0126
5999    19.9825
6000    20.0735
Name: PW_ATT_M_00, Length: 1001, dtype: float64

# Q-Learning Agent

In [ ]:
# Dimensione problema e inizializzazione stato iniziale
[S, feat] = df.shape
current_state = 0 # np.random.randint(0,S-1)

# Stato corrente
s = current_state

min_f = np.min(a_f_m_scaled)
max_f = np.max(a_f_m_scaled)

# Parametri di apprendimento (learning rate, discount factor, average rate)
alpha = 0.5 # non in uso
gamma = 1
beta = 1

# Media mobile esponenziale (avr = avr + beta * (r - avr)) 
# [scelta perché computazionalmente più leggera di una media aritmetica]
average_reward = -np.random.randint(np.min(df.loc[:,'PW_ATT_M_00']),np.max(df.loc[:,'PW_ATT_M_00']))
print("average_reward start: ", average_reward,"\n")

# Setting Episodi
total_episodes = 50000
current_episode = 0

# Ripeti finché non raggiungi la fine del processo
for current_episode in range(total_episodes):
    
    # Stato successivo e ottieni l'azione A e la ricompensa R
    s_prime = s+1
    if s_prime > S-1:
      print("s =", s)
      break

    a1 = df.loc[s,'F_M_FDBK_00']
    a2 = df.loc[s,'VLV_RAF_FDBK_00']
    a = (a1,a2)
    r = -df.loc[s,'PW_ATT_M_00']
    average_reward = average_reward + beta * (r - average_reward)

    # Condizioni iniziali (random) della minimizzazione della funzione obiettivo 
    # per la predizione di q'
    data_f = a_f_m_scaled
    a0_f = data_f[np.random.randint(len(data_f))]
    a0_v = np.random.rand()
    x0 = [a0_f, a0_v]

    # Dominio della minimizzazione 
    bounds = [(min_f, max_f), (0,1)]

    # Risultato della minimizzazione
    objective_minimized = minimize(objective, x0, method = 'Nelder-Mead', bounds = bounds)

    # Compongo input della predizione di q'
    a1_prime_input = objective_minimized.x[0].reshape(1,1)
    a2_prime_input = objective_minimized.x[1].reshape(1,1)
    s_prime_input = pca_df.loc[s_prime,:].values.reshape(7,1)
    nn_input_prime = np.concatenate((s_prime_input, a1_prime_input, a2_prime_input),axis = 0)
    nn_input_prime = np.reshape(nn_input_prime, (1,input_dim))

    # Predizione di q'
    q_prime = rete_neurale.predict(nn_input_prime, verbose = 0)[0]
    q_prime = q_prime[0]
    # Calcola la media delle ricompense future attese per lo stato-azione corrente
    # q = avr + gamma*max(q')
    #print("q' = ", q_prime)
    q = (average_reward + gamma * (q_prime))
    #print("q = ", q)

    # Calcola il valore target per il training della rete neurale
    target = q
    target = np.array([target])

    # Aggiorna la rete neurale
    pca_s = pca_df.loc[s,:]
    nn_input = np.concatenate((pca_s.values, np.array([a[0],a[1]])))
    nn_input = np.reshape(nn_input, (1,input_dim))    
    rete_neurale.fit(nn_input, target, epochs=1, verbose=0)

    # Aggiorna lo stato corrente
    s = s_prime

    # Plot statistiche ogni 100 episodi
    print("current_episode = ", current_episode)
    n_print = 1
    if current_episode%n_print == 0:
      print("Episodio ",current_episode,"/",total_episodes)
      print("target: ", target, "\nreward: ",r,"\naverage_reward: ",np.mean(average_reward))
    print("\n")

# Restituisci la rete neurale ottenuta
#return rete_neurale

average_reward start:  -55 

current_episode =  0
Episodio  0 / 50000
target:  [-25.13659167] 
reward:  -0.0 
average_reward:  -27.5


current_episode =  1
Episodio  1 / 50000
target:  [-11.39047813] 
reward:  -0.0 
average_reward:  -13.75


current_episode =  2
Episodio  2 / 50000
target:  [-4.51780367] 
reward:  -0.0 
average_reward:  -6.875


current_episode =  3
Episodio  3 / 50000
target:  [-1.08338881] 
reward:  -0.0 
average_reward:  -3.4375


current_episode =  4
Episodio  4 / 50000
target:  [0.63226318] 
reward:  -0.0 
average_reward:  -1.71875


current_episode =  5
Episodio  5 / 50000
target:  [1.48847651] 
reward:  -0.0 
average_reward:  -0.859375


current_episode =  6
Episodio  6 / 50000
target:  [1.9150362] 
reward:  -0.0 
average_reward:  -0.4296875


current_episode =  7
Episodio  7 / 50000
target:  [2.12643623] 
reward:  -0.0 
average_reward:  -0.21484375


current_episode =  8
Episodio  8 / 50000
target:  [2.23105073] 
reward:  -0.0 
average_reward:  -0.107421875


c

KeyboardInterrupt: ignored